In [1]:
import pandas as pd
import numpy as np
import torch
import gdown
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import warnings

# --- CONFIGURATION ---
warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. DOWNLOAD DATA
file_ids = {
    'train_split.csv': '1IsyR3Xb_2cfvo457Fh8nE72VTifZb64W',
    'val_split.csv': '1wX4PeKO6fPeLmICA2xmJ3gDUqv_MjE4B',
    'test_split.csv': '1YSyXByVC105ifTZf-FX9IFjlAbZXbdSj',
    'feature_columns.txt': '1PJqBlBZuxocIyHWywBhtaynUn2Pw-y5_'
}
def download_from_drive(ids):
    for filename, fid in ids.items():
        if not os.path.exists(filename):
            url = f'https://drive.google.com/uc?id={fid}'
            gdown.download(url, filename, quiet=False)

download_from_drive(file_ids)

Downloading...
From (original): https://drive.google.com/uc?id=1IsyR3Xb_2cfvo457Fh8nE72VTifZb64W
From (redirected): https://drive.google.com/uc?id=1IsyR3Xb_2cfvo457Fh8nE72VTifZb64W&confirm=t&uuid=3d0f7aef-d2db-432f-9f20-9919059e050d
To: /kaggle/working/train_split.csv
100%|██████████| 1.01G/1.01G [00:05<00:00, 181MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wX4PeKO6fPeLmICA2xmJ3gDUqv_MjE4B
From (redirected): https://drive.google.com/uc?id=1wX4PeKO6fPeLmICA2xmJ3gDUqv_MjE4B&confirm=t&uuid=852e772b-bc05-40b2-91a2-9cc1edc9aaaf
To: /kaggle/working/val_split.csv
100%|██████████| 133M/133M [00:00<00:00, 157MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1YSyXByVC105ifTZf-FX9IFjlAbZXbdSj
From (redirected): https://drive.google.com/uc?id=1YSyXByVC105ifTZf-FX9IFjlAbZXbdSj&confirm=t&uuid=d9ac363d-a302-4e67-be25-633f41f4167b
To: /kaggle/working/test_split.csv
100%|██████████| 133M/133M [00:00<00:00, 204MB/s] 
Downloading...
From: https://drive.google.

In [2]:
# 2. DATA LOADING & PREPARATION
# Load full data and sort by date to ensure temporal integrity
df_train = pd.read_csv('train_split.csv')
df_val = pd.read_csv('val_split.csv')
df = pd.concat([df_train, df_val], ignore_index=True)

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date') 

with open('feature_columns.txt', 'r') as f:
    features = [line.strip() for line in f.readlines()]

In [3]:
# 3. SCRATCH LINEAR REGRESSION (CUDA)
class LinearRegressionScratch:
    def fit(self, X, y):
        # Adding bias term
        ones = torch.ones((X.shape[0], 1), device=device)
        X_b = torch.cat((ones, X), dim=1)
        # Normal Equation using Pseudo-inverse for numerical stability
        self.weights = torch.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y
        
    def predict(self, X):
        ones = torch.ones((X.shape[0], 1), device=device)
        X_b = torch.cat((ones, X), dim=1)
        # Clamp at 0 because sales cannot be negative
        return torch.clamp(X_b @ self.weights, min=0)

# 4. SCRATCH GRADIENT BOOSTED REGRESSOR (CUDA)
class GBDTScratch:
    def __init__(self, n_estimators=10, max_depth=3, lr=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.lr = lr
        self.trees = []

    def fit(self, X, y):
        # Initial prediction (mean of target)
        self.base_pred = torch.mean(y)
        current_preds = torch.full_like(y, self.base_pred)

        for i in range(self.n_estimators):
            # Gradient Boosting: Predict the residuals (errors)
            residuals = y - current_preds
            
            # Build tree on residuals
            tree = self._build_tree(X, residuals, depth=0)
            self.trees.append(tree)
            
            # Update overall predictions with learning rate
            current_preds += self.lr * self._predict_tree(X, tree)
            
            # Requirement: Report RMSE at each boosting stage
           
            rmse = torch.sqrt(torch.mean((y - current_preds)**2))
            print(f"  > Boosting Round {i}, Residual RMSE: {rmse.item():.2f}")

    def _build_tree(self, X, res, depth):
        if depth >= self.max_depth or len(res) < 10:
            return torch.mean(res)

        n_features = X.shape[1]
        best_gain = -1
        split_idx, split_val = 0, 0

        # Optimization: Median-split on sampled features for speed
        for f_idx in range(n_features):
            feat_vals = X[:, f_idx]
            threshold = torch.median(feat_vals)
            
            left_mask = feat_vals <= threshold
            right_mask = ~left_mask
            
            if left_mask.sum() == 0 or right_mask.sum() == 0: continue
            
            # Variance Reduction Gain
            gain = torch.var(res) - (left_mask.sum()*torch.var(res[left_mask]) + 
                                     right_mask.sum()*torch.var(res[right_mask])) / len(res)
            
            if gain > best_gain:
                best_gain, split_idx, split_val = gain, f_idx, threshold

        left_mask = X[:, split_idx] <= split_val
        return {
            'split_idx': split_idx,
            'split_val': split_val,
            'left': self._build_tree(X[left_mask], res[left_mask], depth+1),
            'right': self._build_tree(X[~left_mask], res[~left_mask], depth+1)
        }

    def _predict_tree(self, X, tree):
        if not isinstance(tree, dict):
            return torch.full((X.shape[0],), tree, device=device)
        
        preds = torch.zeros(X.shape[0], device=device)
        left_mask = X[:, tree['split_idx']] <= tree['split_val']
        
        preds[left_mask] = self._predict_tree(X[left_mask], tree['left'])
        preds[~left_mask] = self._predict_tree(X[~left_mask], tree['right'])
        return preds

    def predict(self, X):
        y_pred = torch.full((X.shape[0],), self.base_pred, device=device)
        for tree in self.trees:
            y_pred += self.lr * self._predict_tree(X, tree)
        return torch.clamp(y_pred, min=0)

# 5. EVALUATION: STORE-AWARE TEMPORAL CROSS-VALIDATION
def get_rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred)**2)).item()


In [4]:
for column in df[features].columns:
    if df[column].dtype == 'object':
        print(f"Object Column Found: {column}")
        # Show a few unique values to see what the data looks like
        print(f"Sample values: {train_df[column].unique()[:5]}\n")

In [5]:
tscv = TimeSeriesSplit(n_splits=5)
fold_results_lr = []
for fold, (train_idx, val_idx) in enumerate(tscv.split(df)):
    print(f"\n--- FOLD {fold + 1} / 5 ---")
    
    # Split data based on time-indices
    train_fold = df.iloc[train_idx]
    val_fold = df.iloc[val_idx]
    # 1. Force all columns to numeric and cast the final array to float32
    X_train_numeric = train_fold[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
    X_val_numeric = val_fold[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

    # 2. Now convert the pure float32 array to a Tensor
    X_train = torch.tensor(X_train_numeric, device=device)
    y_train = torch.tensor(train_fold['Sales'].values.astype(np.float32), device=device)
    
    X_val = torch.tensor(X_val_numeric, device=device)
    y_val = torch.tensor(val_fold['Sales'].values.astype(np.float32), device=device)

    # Train Scratch Linear Regression
    lr_model = LinearRegressionScratch()
    lr_model.fit(X_train, y_train)
    lr_rmse = get_rmse(y_val, lr_model.predict(X_val))
    fold_results_lr.append(lr_rmse)
    
print(f"Average Linear Regression RMSE: {np.mean(fold_results_lr):.2f}")


--- FOLD 1 / 5 ---

--- FOLD 2 / 5 ---

--- FOLD 3 / 5 ---

--- FOLD 4 / 5 ---

--- FOLD 5 / 5 ---
Average Linear Regression RMSE: 752.02


In [6]:
import itertools

# 1. Define the Hyperparameter Grid
param_grid = {
    'max_depth': [3, 4, 5, 10],
    'lr': [0.05, 0.1],
    'n_estimators':[5, 10]
}

# Generate all combinations
keys, values = zip(*param_grid.items())
permutations = [dict(zip(keys, v)) for v in itertools.product(*values)]

best_overall_rmse = float('inf')
best_config = None

print(f"Starting Grid Search over {len(permutations)} combinations...")

# --- NEW OUTER LOOP FOR GRID SEARCH ---
for params in permutations:
    print(f"\n" + "#"*60)
    print(f"TESTING CONFIGURATION: {params}")
    print("#"*60)

    # Defining the Cross-Validation strategy
    # This ensures we test on multiple future windows of the stores
    tscv = TimeSeriesSplit(n_splits=5)
    fold_results_gbdt = []

    print("Starting Store-Aware Temporal Cross-Validation...")

    for fold, (train_idx, val_idx) in enumerate(tscv.split(df)):
        print(f"\n--- FOLD {fold + 1} / 5 ---")
        
        # Split data based on time-indices
        train_fold = df.iloc[train_idx]
        val_fold = df.iloc[val_idx]
        
        # 1. Force all columns to numeric and cast the final array to float32
        X_train_numeric = train_fold[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
        X_val_numeric = val_fold[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

        # 2. Now convert the pure float32 array to a Tensor
        X_train = torch.tensor(X_train_numeric, device=device)
        y_train = torch.tensor(train_fold['Sales'].values.astype(np.float32), device=device)
        
        X_val = torch.tensor(X_val_numeric, device=device)
        y_val = torch.tensor(val_fold['Sales'].values.astype(np.float32), device=device)

        # Train Custom GBDT (Using parameters from Grid Search)
        gbdt_model = GBDTScratch(n_estimators=params["n_estimators"], max_depth=params['max_depth'], lr=params['lr'])
        gbdt_model.fit(X_train, y_train)
        gbdt_rmse = get_rmse(y_val, gbdt_model.predict(X_val))
        fold_results_gbdt.append(gbdt_rmse)

        print(f"Fold Results -> LR RMSE: {lr_rmse:.2f} | GBDT RMSE: {gbdt_rmse:.2f}")

    # Calculate Average for this config
    avg_gbdt_rmse = np.mean(fold_results_gbdt)
    
    # Track the best configuration
    if avg_gbdt_rmse < best_overall_rmse:
        best_overall_rmse = avg_gbdt_rmse
        best_config = params

    # 6. SUMMARY FOR CURRENT CONFIGURATION
    print("\n" + "="*40)
    print(f"SUMMARY FOR CONFIG: {params}")
    print("="*40)
    print(f"Average Custom GBDT RMSE: {avg_gbdt_rmse:.2f}")
    print("="*40)

# FINAL GLOBAL RESULT
print("\n\n" + "!"*60)
print(f"GRID SEARCH COMPLETE")
print(f"BEST CONFIGURATION For gdb: {best_config}")
print(f"LOWEST AVERAGE CV RMSE: {best_overall_rmse:.2f}")
print("!"*60)

Starting Grid Search over 16 combinations...

############################################################
TESTING CONFIGURATION: {'max_depth': 3, 'lr': 0.05, 'n_estimators': 5}
############################################################
Starting Store-Aware Temporal Cross-Validation...

--- FOLD 1 / 5 ---
  > Boosting Round 0, Residual RMSE: 3594.27
  > Boosting Round 1, Residual RMSE: 3452.72
  > Boosting Round 2, Residual RMSE: 3319.41
  > Boosting Round 3, Residual RMSE: 3194.08
  > Boosting Round 4, Residual RMSE: 3075.77
Fold Results -> LR RMSE: 684.57 | GBDT RMSE: 2974.50

--- FOLD 2 / 5 ---
  > Boosting Round 0, Residual RMSE: 3531.00
  > Boosting Round 1, Residual RMSE: 3392.42
  > Boosting Round 2, Residual RMSE: 3262.19
  > Boosting Round 3, Residual RMSE: 3139.62
  > Boosting Round 4, Residual RMSE: 3024.36
Fold Results -> LR RMSE: 684.57 | GBDT RMSE: 3396.66

--- FOLD 3 / 5 ---
  > Boosting Round 0, Residual RMSE: 3659.28
  > Boosting Round 1, Residual RMSE: 3516.12
  > B

In [5]:
#now, we train a model using the best hyperparameters that we found and test it using the test dataset
# 1. Force all columns to numeric and cast the final array to float32
test_df = pd.read_csv("test_split.csv")
X_train_numeric = df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
X_test_numeric = test_df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

# 2. Now convert the pure float32 array to a Tensor
X_train = torch.tensor(X_train_numeric, device=device)
y_train = torch.tensor(df['Sales'].values.astype(np.float32), device=device)

X_test = torch.tensor(X_test_numeric, device=device)
y_test = torch.tensor(test_df['Sales'].values.astype(np.float32), device=device)

# Train Custom GBDT (Using best parameters)
gbdt_model = GBDTScratch(n_estimators=10, max_depth=10, lr=0.1)
gbdt_model.fit(X_train, y_train)
gbdt_rmse = get_rmse(y_test, gbdt_model.predict(X_test))

print(f"final gbdt rmse = {gbdt_rmse}")

  > Boosting Round 0, Residual RMSE: 3480.07
  > Boosting Round 1, Residual RMSE: 3153.91
  > Boosting Round 2, Residual RMSE: 2861.62
  > Boosting Round 3, Residual RMSE: 2599.03
  > Boosting Round 4, Residual RMSE: 2362.60
  > Boosting Round 5, Residual RMSE: 2151.61
  > Boosting Round 6, Residual RMSE: 1962.60
  > Boosting Round 7, Residual RMSE: 1793.67
  > Boosting Round 8, Residual RMSE: 1642.41
  > Boosting Round 9, Residual RMSE: 1506.33
final gbdt rmse = 1549.720458984375


In [7]:
#now, we increase the number of estimators to 50
# 1. Force all columns to numeric and cast the final array to float32
test_df = pd.read_csv("test_split.csv")
X_train_numeric = df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
X_test_numeric = test_df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

# 2. Now convert the pure float32 array to a Tensor
X_train = torch.tensor(X_train_numeric, device=device)
y_train = torch.tensor(df['Sales'].values.astype(np.float32), device=device)

X_test = torch.tensor(X_test_numeric, device=device)
y_test = torch.tensor(test_df['Sales'].values.astype(np.float32), device=device)

# Train Custom GBDT
gbdt_model = GBDTScratch(n_estimators=50, max_depth=10, lr=0.1)
gbdt_model.fit(X_train, y_train)
gbdt_rmse = get_rmse(y_test, gbdt_model.predict(X_test))

print(f"final gbdt test rmse = {gbdt_rmse}")

  > Boosting Round 0, Residual RMSE: 3480.07
  > Boosting Round 1, Residual RMSE: 3153.91
  > Boosting Round 2, Residual RMSE: 2861.62
  > Boosting Round 3, Residual RMSE: 2599.03
  > Boosting Round 4, Residual RMSE: 2362.60
  > Boosting Round 5, Residual RMSE: 2151.61
  > Boosting Round 6, Residual RMSE: 1962.60
  > Boosting Round 7, Residual RMSE: 1793.67
  > Boosting Round 8, Residual RMSE: 1642.41
  > Boosting Round 9, Residual RMSE: 1506.33
  > Boosting Round 10, Residual RMSE: 1384.88
  > Boosting Round 11, Residual RMSE: 1277.50
  > Boosting Round 12, Residual RMSE: 1180.80
  > Boosting Round 13, Residual RMSE: 1094.31
  > Boosting Round 14, Residual RMSE: 1018.64
  > Boosting Round 15, Residual RMSE: 951.08
  > Boosting Round 16, Residual RMSE: 891.85
  > Boosting Round 17, Residual RMSE: 838.30
  > Boosting Round 18, Residual RMSE: 793.00
  > Boosting Round 19, Residual RMSE: 752.82
  > Boosting Round 20, Residual RMSE: 716.70
  > Boosting Round 21, Residual RMSE: 685.72
  > B

In [9]:
#training and testing the base line
# 1. Force all columns to numeric and cast the final array to float32
test_df = pd.read_csv("test_split.csv")
X_train_numeric = df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)
X_test_numeric = test_df[features].apply(pd.to_numeric, errors='coerce').fillna(0).values.astype(np.float32)

# 2. Now convert the pure float32 array to a Tensor
X_train = torch.tensor(X_train_numeric, device=device)
y_train = torch.tensor(df['Sales'].values.astype(np.float32), device=device)

X_test = torch.tensor(X_test_numeric, device=device)
y_test = torch.tensor(test_df['Sales'].values.astype(np.float32), device=device)
# Train Scratch Linear Regression
lr_model = LinearRegressionScratch()
lr_model.fit(X_train, y_train)
lr_rmse = get_rmse(y_test, lr_model.predict(X_test))
print(f"final base line (regression) test rmse = {lr_rmse}")

final base line (regression) test rmse = 1330.7149658203125


In [10]:
torch.save(gbdt_model, "gbdt_model.pt")